In [ ]:
path = []


In [2]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [19]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

[[5.0, 5.0], [7.5, 7.5], [10.5, 10.5]]


In [32]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

In [33]:
import math
print(slp)

[1.1292184306130484, 0.0007963267948964958]


In [84]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [ ]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost


In [112]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memry = deque(maxlen=pow(10,6))
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = None
        self.name = 'steerDeepQL.pkl'
    
    def xgbmodel_bld(self,data,label):
        param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 6}  
        param['nthread'] = 4
        num_round = 2000  
#         data = np.random.rand(5, 10)  
#         label = np.random.randint(2, size=5)  
        dtrain = xgb.DMatrix(data, label=label)
        self.bst = xgb.train(param, dtrain, num_round)
        self.dump_bst(name)
        
    def load_bst():
        booster = joblib.load(name)
        return booster
    
    def dump_bst():
        joblib.save_model(bst, name, compress=True)
        
    def act(state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(state)
        return np.argmax(act_values[0])         

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.bst.predict(state)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
        self.xgbmodel_bld(state, target)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1]))
        d = False
        if rew > 130:
            d = True
        return [rew, True]

    
    def get_nextstate():
        ########
        return stt
    
def steer(GameOn):
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((1,15),ndtype = 'float')
    state = get_nextstate()
    done = False
    while GameOn:
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
        print("action:",action)
        execute_action(action)
        reward, done = decl_rew(state)
        reward = reward if not done else -10
        next_state = get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("updating weights under 10 deg")
            agent.update_target_model()
        if len(agent.memory) > batch_size:
            print("replaying at 50 and saving weights")
            agent.replay(batch_size)
    
        

In [89]:
import random
import gym
import numpy as np
from collections import deque
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam
from keras import backend as K

EPISODES = 500


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        
    
    def _build_model(self):
        param = { 'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 6}  
        num_round = 2000  
        classifier = xgb.sklearn.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
                        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
                       objective='multi:softmax', reg_alpha=0, reg_lambda=1,
                       scale_pos_weight=1, seed=42, silent=True, subsample=1)
        return classifier
    
    def update_target_model(self):
        self.target_model.load_model(self.model.save_model())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) 
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
#         eval_set = [(X_test, y_test)]
        self.model.fit(state, target,eval_metric="error", eval_set=eval_set, verbose=True)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_model(name)
    
    def save(self, name):
        self.model.save_model(name)
    
    def decl_rew(self,state):
        rew = np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1]))
        d = False
        if rew > 130:
            d = True
        return [nxtstate, rew, True]
        


def steer():
    state_size = 15
    action_size = 6
    agent = DQNAgent(state_size, action_size)
    
#     agent.load_model("./save/steerwts.h5")
    
    batch_size = 50
    state = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] 
    while True:
        done = False
        while True:
            state = np.reshape(state, [1, state_size])
            action = agent.act(state)
            print("action:",action)
            execute_action(action)
            next_state, reward, done = decl_rew(state)##input and reward
            if(next_state == []):
                continue;
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("updating weights under 10 deg")
                agent.update_target_model()
            if len(agent.memory) > batch_size:
                print("replaying at 50 and saving weights")
                agent.replay(batch_size)

In [90]:
steer()

AttributeError: 'XGBClassifier' object has no attribute 'load_model'

In [49]:
state = env.reset()
print(state_size)
state = np.reshape(state, [-1, state_size])
print(state)

4
[[-0.02340593 -0.04727202 -0.00067567 -0.02899503]]


In [50]:
# agent.save("./save/cartpole-ddqn.h5")
print(state_size)

4


In [12]:
import numpy as np
from PIL import ImageGrab
import cv2
import time

def screen_record(): 
    last_time = time.time()
    while True:
        # 800x600 windowed mode
        printscreen_pil =  ImageGrab.grab(bbox=(0,40,800,640))
        printscreen_numpy =   np.array(printscreen_pil.getdata(),dtype='uint8')\
        .reshape((printscreen_pil.size[1],printscreen_pil.size[0],3))
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()

    ##    cv2.imshow('window',cv2.cvtColor(printscreen_numpy, cv2.COLOR_BGR2RGB))
    ##    if cv2.waitKey(25) & 0xFF == ord('q'):
    ##        cv2.destroyAllWindows()
    ##        break

In [52]:
import math
print(pow(10,100))

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [102]:
data = np.random.rand(5,10) # 5 entities, each contains 10 features
label = np.random.randint(2, size=5) # binary target
dtest = np.random.rand(5,10)
dtrain = xgb.DMatrix( data, label=label)
print(dtrain)

In [103]:
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'multi:softmax' }
param['nthread'] = 4
plst = param.items() 
# plst += [('eval_metric', 'merror')] 
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [104]:
num_round = 10

bst = xgb.train( plst, dtrain, num_round, evallist )

TypeError: invalid cache item: ndarray

In [115]:
import numpy as np
import xgboost as xgb
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')
param = {
    'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 6}  
num_round = 2000  
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

best_preds = np.asarray([np.argmax(line) for line in preds])
print("Numpy array precision:", precision_score(y_test, best_preds, average='macro') )

# bst_svm = xgb.train(param, dtrain_svm, num_round)
# preds = bst.predict(dtest_svm)

# best_preds_svm = [np.argmax(line) for line in preds]
# print("Svm file precision:",precision_score(y_test, best_preds_svm, average='macro'))
# bst.dump_model('dump.raw.txt')
# bst_svm.dump_model('dump_svm.raw.txt')

joblib.save_model(bst, 'bst_model.pkl', compress=True)
# joblib.save(bst_svm, 'bst_svm_model.pkl', compress=True)


Numpy array precision: 1.0


AttributeError: module 'sklearn.externals.joblib' has no attribute 'save_model'

In [117]:
xgb1 = joblib.load('bst_model.pkl')
xgb1.train(param, dtrain, num_round)
# xgb1.predict(dtest)
# joblib.dump(xgb1, 'bst_modell.pkl', compress=True)

AttributeError: 'Booster' object has no attribute 'train'